In [3]:
# 구글 드라이브에 저장된 파일을 사용하기 위한 코드
from google.colab import drive
drive.mount('/content/drive')

# 저장된 파일의 경로 설정
%cd /content/drive/MyDrive/LG 오프라인

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/LG 오프라인


## 가중치 리스트를 문자열로 저장한 데이터 구조

In [22]:
import pandas as pd
import numpy as np

# 데이터 불러오기
data_path = 'train.csv'
df = pd.read_csv(data_path)

# 날짜 컬럼을 datetime 형식으로 변환하고 요일 정보를 얻습니다.
date_columns = df.columns[7:]
date_to_weekday = pd.to_datetime(date_columns).to_series().dt.dayofweek

# "제품" 컬럼을 기준으로 요일별 판매량 합계를 계산합니다.
product_sales_by_day = df.groupby("제품")[date_columns].sum().transpose()

# 요일별 판매량 합계를 얻기 위해 같은 요일끼리 그룹화하고 합계를 계산합니다.
sales_sum_by_weekday = product_sales_by_day.groupby(date_to_weekday).sum().transpose()

# 학습 데이터 내 각 요일의 수를 계산합니다.
weekday_counts = date_to_weekday.value_counts().sort_index()

# 요일별 판매량 평균을 계산합니다.
sales_avg_by_weekday = sales_sum_by_weekday.divide(weekday_counts, axis=1)

# NaN 값을 0으로 대체합니다.
sales_avg_by_weekday.fillna(0, inplace=True)

# 각 제품별로 가장 낮은 판매량 평균을 찾습니다.
min_sales_avg_by_weekday = sales_avg_by_weekday.min(axis=1)

# 가장 낮은 판매량 평균을 1로 두고 나머지 요일들의 가중치를 계산합니다.
sales_weight_by_weekday = sales_avg_by_weekday.divide(min_sales_avg_by_weekday, axis=0)

# 가중치가 무한대(inf)로 계산되는 경우를 방지하기 위해 NaN과 inf를 0으로 대체합니다.
sales_weight_by_weekday.replace([np.inf, -np.inf], np.nan, inplace=True)
sales_weight_by_weekday.fillna(0, inplace=True)

# 계산된 7일의 가중치를 리스트로 저장하고 문자열로 변환하여 새로운 컬럼에 저장합니다.
sales_weight_by_weekday['weights'] = sales_weight_by_weekday.apply(lambda row: str(row.tolist()), axis=1)

# '제품' 컬럼을 기준으로 원본 데이터프레임과 가중치 데이터프레임을 병합합니다.
result_df = df.merge(sales_weight_by_weekday['weights'], on='제품')

result_df = result_df[['ID','weights']]

# 결과를 CSV 파일로 저장합니다.
result_df.to_csv('train_with_weights.csv', index=False)


## 가중치를 저장한 train 데이터

In [29]:
import pandas as pd
import numpy as np

# CSV 파일 로드
weights_data_path = 'train_with_weights.csv'
train_data_path = 'train.csv'

weights_data = pd.read_csv(weights_data_path)
train_data = pd.read_csv(train_data_path)

# 문자열 형태의 가중치를 리스트로 변환
import ast
weights_data['weights'] = weights_data['weights'].apply(ast.literal_eval)

# 요일별 가중치 리스트를 numpy 배열로 변환
weights_data['weights'] = weights_data['weights'].apply(np.array)

# 요일별 가중치를 열로 가지는 새 데이터프레임 생성
days_of_week = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
weights_df = weights_data['weights'].apply(pd.Series)
weights_df.columns = days_of_week
weights_df['ID'] = weights_data['ID']
weights_df = weights_df[['ID'] + days_of_week]

# 'ID' 열을 기준으로 train_data와 weights_df를 병합
merged_data = pd.merge(train_data, weights_df, on="ID")

# 판매량 데이터의 날짜 열을 가져옴
date_columns = train_data.columns[7:]

# 요일 인덱스를 생성 (0:월요일, 1:화요일, ..., 6:일요일)
start_date = pd.to_datetime(date_columns[0])
day_indices = (start_date + pd.to_timedelta(np.arange(len(date_columns)), unit='D')).dayofweek

# 일별 판매량 데이터와 요일 가중치 데이터를 numpy 배열로 변환
sales_data = merged_data[date_columns].values
weights_data = merged_data[days_of_week].values

# 요일 인덱스를 사용하여 각 판매량에 적절한 가중치를 선택
adjusted_sales_data = sales_data * weights_data[:, day_indices]

# 새로운 판매량 데이터프레임을 생성
new_train_data = merged_data.copy()
new_train_data[date_columns] = adjusted_sales_data

# 'ID', '제품', '월요일'부터 '일요일'까지의 컬럼만을 포함하는 데이터프레임 생성
final_train_data = new_train_data[['ID', '제품'] + days_of_week]

# 결과를 새 CSV 파일로 저장
output_csv_path = 'weights(date).csv'
final_train_data.to_csv(output_csv_path, index=False)


In [30]:
final_train_data

,ID,제품,월요일,화요일,수요일,목요일,금요일,토요일,일요일
0,SAMPLE_00000,B002-00001-00001,1.000000,1.490349,2.219669,1.965993,1.997702,1.156250,1.000000
1,SAMPLE_00001,B002-00002-00001,1.028986,1.000000,1.161765,1.433824,1.580882,1.391304,1.159420
2,SAMPLE_00002,B002-00002-00002,1.134827,1.121212,1.000000,1.068182,1.098485,1.231884,1.149758
3,SAMPLE_00003,B002-00002-00003,1.231884,1.372340,1.053191,1.117021,1.000000,1.300031,1.315757
4,SAMPLE_00004,B002-00002-00004,2.800000,1.862896,2.976471,1.436199,2.133484,1.543590,1.000000
...,...,...,...,...,...,...,...,...,...
28889,SAMPLE_28889,B002-03798-00046,1.000000,1.070306,1.244057,1.306608,1.369158,1.349315,1.184932
28890,SAMPLE_28890,B002-03799-00002,2.526731,1.108333,1.000000,1.747222,2.555556,3.389050,3.835266
28891,SAMPLE_28891,B002-03799-00003,1.000837,1.000000,1.266667,1.551111,1.562222,1.629372,1.314010
28892,SAMPLE_28892,B002-03799-00004,1.081868,1.000000,1.088889,1.235556,1.320000,1.112528,1.038068
